## Libraries

In [ ]:
%%writefile GLONASS_FCN_OSN_MAP.json
{
  "test_2020-08-03-US-MTV-2_Mi8": {
    "96": -1,
    "98": 13,
    "103": 19,
    "104": -1,
    "105": 3
  },
  "test_2020-08-13-US-MTV-1_Mi8": {
    "105": 7,
    "106": 4,
    "103": 19
  },
  "test_2021-03-25-US-PAO-1_Mi8": {
    "98": 9,
    "102": 24
  },
  "train_2020-05-21-US-MTV-2_Pixel4": {
    "96": -1
  },
  "train_2020-05-21-US-MTV-2_Pixel4XL": {
    "96": -1
  },
  "train_2020-05-29-US-MTV-1_Pixel4": {
    "103": 23
  },
  "train_2020-07-17-US-MTV-1_Mi8": {
    "106": -1,
    "99": 16,
    "100": 15,
    "103": 23
  },
  "train_2020-07-17-US-MTV-2_Mi8": {
    "106": -1,
    "99": -1,
    "101": 1,
    "103": -1
  },
  "train_2020-08-03-US-MTV-1_Mi8": {
    "98": 9,
    "99": -1,
    "102": -1,
    "103": 19
  },
  "train_2020-08-06-US-MTV-2_Mi8": {
    "105": 7,
    "102": 20,
    "103": -1
  },
  "train_2020-09-04-US-SF-1_Mi8": {
    "105": 3,
    "106": 8,
    "93": 14,
    "103": 23
  },
  "train_2020-09-04-US-SF-2_Mi8": {
    "106": -1,
    "100": 15,
    "101": 1
  },
  "train_2021-01-05-US-SVL-1_Mi8": {
    "104": 21,
    "97": 22
  },
  "train_2021-04-26-US-SVL-1_Mi8": {
    "106": 4
  },
  "train_2021-04-28-US-SJC-1_Pixel4": {
    "93": -1
  }
}

In [ ]:
%%writefile generate_phone_isrbM.py
import json
from dataclasses import dataclass
from collections import defaultdict
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

INPUT_PATH = '../input/google-smartphone-decimeter-challenge'

@dataclass
class Average:
    total: float = 0
    count: float = 0
    def __add__(self, other):
        total = self.total + other.total
        count = self.count + other.count
        return Average(total=total, count=count)
    @property
    def value(self):
        return self.total / self.count

def main():
    path_list = glob.glob(f'{INPUT_PATH}/train/*/*') + glob.glob(f'{INPUT_PATH}/test/*/*')
    phone_to_drive = defaultdict(list)
    for path in path_list:
        prefix, drive, phone = path.split('/')[-3:]
        phone_to_drive[phone].append((prefix, drive))

    PHONE_ISRBM = dict()
    for phone in tqdm(phone_to_drive.keys()):
        phone_isrbM = defaultdict(Average)
        for prefix, drive in phone_to_drive[phone]:
            df = pd.read_csv(f'{INPUT_PATH}/{prefix}/{drive}/{phone}/{phone}_derived.csv')
            for sig_type, sig_df in df.groupby('signalType'):
                isrbM = sig_df['isrbM'].values
                q10   = np.quantile(isrbM, 0.10)
                q90   = np.quantile(isrbM, 0.90)
                cond  = (q10 <= isrbM) & (isrbM <= q90)
                isrbM = isrbM[cond]
                mu = np.mean(isrbM)
                N  = min(5000, isrbM.shape[0])
                phone_isrbM[sig_type] += Average(total=mu * N, count=N)
        PHONE_ISRBM[phone] = {sig : ave.value for sig, ave in phone_isrbM.items()}
    with open('phone_isrbM.json', 'w') as f:
        json.dump(PHONE_ISRBM, f)
    return

In [ ]:
%%writefile constants.py
import json
import datetime
from collections import defaultdict
import numpy as np

with open('GLONASS_FCN_OSN_MAP.json') as f:
    GLONASS_FCN_OSN_MAP = json.load(f)
with open('phone_isrbM.json') as f:
    PHONE_ISRBM = json.load(f)
del f

GPS_ORIGIN_DAY       = datetime.date(1980, 1, 6)
GPS_ORIGIN_DATETIME  = datetime.datetime(1980, 1, 6)
GLONASS_LEAP_SECONDS = 18
BEIDOU_LEAP_SECONDS  = 14
TZ_MSK = datetime.timezone(datetime.timedelta(hours=+3), 'MSK')

WGS84_SEMI_MAJOR_AXIS = 6378137.0
WGS84_SEMI_MINOR_AXIS = 6356752.314245
WGS84_SQUARED_FIRST_ECCENTRICITY  = 6.69437999013e-3
WGS84_SQUARED_SECOND_ECCENTRICITY = 6.73949674226e-3
WGS84_FIRST_ECCENTRICITY  = np.sqrt(WGS84_SQUARED_FIRST_ECCENTRICITY)
WGS84_SECOND_ECCENTRICITY = np.sqrt(WGS84_SQUARED_SECOND_ECCENTRICITY)

LIGHT_SPEED = 299792458.0

OMEGA_EARTH = 7.2921151467e-5
MU_EARTH    = 3.986005e+14

FREQ_GPS_L1  = 1.575420e+09
FREQ_GPS_L5  = 1.176450e+09
FREQ_GAL_E1  = FREQ_GPS_L1
FREQ_GAL_E5A = FREQ_GPS_L5
FREQ_QZS_J1  = FREQ_GPS_L1
FREQ_QZS_J5  = FREQ_GPS_L5
FREQ_BDS_B1I = 1.561098e+09
FREQ_GLO_G1_NOMINAL = 1602.00 * 1e+6
FREQ_GLO_G1_DELTA   = 562.5 * 1e+3

CONSTELLATION_TYPE_MAP = {
    'GPS'     : 1,
    'GLONASS' : 3,
    'QZSS'    : 4,
    'BEIDOU'  : 5,
    'GALILEO' : 6,
}

RAW_STATE_BIT_MAP = {
     0: "Code Lock",
     1: "Bit Sync",
     2: "Subframe Sync",
     3: "Time Of Week Decoded State",
     4: "Millisecond Ambiguity",
     5: "Symbol Sync",
     6: "GLONASS String Sync",
     7: "GLONASS Time Of Day Decoded",
     8: "BEIDOU D2 Bit Sync",
     9: "BEIDOU D2 Subframe Sync",
    10: "Galileo E1BC Code Lock",
    11: "Galileo E1C 2^nd^ Code Lock",
    12: "Galileo E1B Page Sync",
    13: "SBAS Sync",
    14: "Time Of Week Known",
    15: "GLONASS Time Of Day Known",
}
RAW_STATE_BIT_INV_MAP = { value : key for key, value in RAW_STATE_BIT_MAP.items() }

ACCUMULATED_DELTA_RANGE_STATE_BIT_MAP = {
    0 : 'VALID',
    1 : 'RESET',
    2 : 'CYCLE_SLIP',
    3 : 'HALF_CYCLE_RESOLVED',
    4 : 'HALF_CYCLE_REPORTED',
}
ACCUMULATED_DELTA_RANGE_STATE_BIT_INT_MAP = {
    value : key for key, value
    in ACCUMULATED_DELTA_RANGE_STATE_BIT_MAP.items()
}

SYSTEM_NAME_MAP = {
    'GPS'     : 'G',
    'GLONASS' : 'R',
    'GALILEO' : 'E',
    'BEIDOU'  : 'C',
    'QZSS'    : 'J',
}

GLONASS_FREQ_CHANNEL_MAP = {
    1 : 1,
    2 : -4,
    3 : 5,
    4 : 6,
    5 : 1,
    6 : -4,
    7 : 5,
    8 : 6,
    9 : -2,
    10 : -7,
    11 : 0,
    12 : -1,
    13 : -2,
    14 : -7,
    15 : 0,
    16 : -1,
    17 : 4,
    18 : -3,
    19 : 3,
    20 : 2,
    21 : 4,
    22 : -3,
    23 : 3,
    24 : 2,
}

QZSS_PRN_SVID_MAP = {
    193 : 1,
    194 : 2,
    199 : 3,
    195 : 4,
}

INIT_B = np.deg2rad(  37.5)
INIT_L = np.deg2rad(-122.2)
INIT_H = 0.0

FREQ_TOL = 100.0
Cn0DbHz_THRESHOLD = 20.0
ReceivedSvTimeUncertaintyNanos_THRESHOLD = 500
RAW_PSEUDO_RANGE_THRESHOLD = 50_000 * 1e+3

CLOCK_TIME_MARGIN = datetime.timedelta(seconds=90)
ORBIT_TIME_MARGIN = datetime.timedelta(hours=3)
IONO_TIME_MARGIN  = datetime.timedelta(hours=2)

EPSILON_M = 0.01
DEFAULT_TROPO_DELAY_M = 2.48

HAVERSINE_RADIUS = 6_371_000

In [ ]:
%%writefile io_f.py
import io
import datetime
from dataclasses import dataclass, asdict
import numpy as np
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline, RectBivariateSpline

@dataclass
class IONEX:
    iono_height : float
    base_radius : float
    lat_1       : float
    lat_2       : float
    lat_delta   : float
    lng_1       : float
    lng_2       : float
    lng_delta   : float
    time_1      : np.datetime64
    time_2      : np.datetime64
    time_delta  : np.timedelta64
    iono_map    : np.array
    lat_range   : np.array
    lng_range   : np.array

def concat_clk(clk_df_list):
    clk_df = pd.concat(clk_df_list, axis=0)
    sat_set_list = [frozenset(clk_df['SatName']) for clk_df in clk_df_list]
    sat_prod = sat_set_list[0]
    for sat_set in sat_set_list[1:]:
        sat_prod = sat_prod & sat_set
    clk_df = clk_df[clk_df['SatName'].isin(sat_prod)]
    clk_df = clk_df.reset_index(drop=True)
    return clk_df

def concat_sp3(sp3_df_list):
    sp3_df  = pd.concat(sp3_df_list, axis=0)
    sat_set_list = [frozenset(sp3_df['SatName']) for sp3_df in sp3_df_list]
    sat_prod = sat_set_list[0]
    for sat_set in sat_set_list[1:]:
        sat_prod = sat_prod & sat_set
    sp3_df = sp3_df[sp3_df['SatName'].isin(sat_prod)]
    sp3_df = sp3_df.reset_index(drop=True)
    return sp3_df

def concat_ionex(ionex_list):
    assert(len(np.unique([ionex.iono_height for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.base_radius for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.lat_1       for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.lat_2       for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.lat_delta   for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.lng_1       for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.lng_2       for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.lng_delta   for ionex in ionex_list])) == 1)
    assert(len(np.unique([ionex.time_delta  for ionex in ionex_list])) == 1)
    N = len(ionex_list)
    iono_map = []
    for i in range(N-1):
        assert(ionex_list[i].time_2 == ionex_list[i+1].time_1)
        iono_map.append(ionex_list[i].iono_map[0:-1, :, :])
    iono_map.append(ionex_list[-1].iono_map)
    kw = asdict(ionex_list[0])
    kw['time_2']   = ionex_list[-1].time_2
    kw['iono_map'] = np.concatenate(iono_map, axis=0)
    return IONEX(**kw)

def read_GnssLog_Raw(filename):
    lines = []
    with open(filename, 'r') as f:
        for line in f:
            if 'Raw' in line:
                line = line.rstrip().lstrip('#')
                lines.append(line)
    sio = io.StringIO('\n'.join(lines))
    return pd.read_csv(sio)

def read_clock_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    for index, line in enumerate(lines):
        if 'TIME SYSTEM ID' in line:
            assert(line.strip().split()[0] == 'GPS')
            continue
        if 'END OF HEADER' in line:
            start_index = index + 1
            break
    lines = lines[start_index:]
    SAT, EPOCH, DELTA_TSV = [], [], []
    for line in lines:
        if not line.startswith('AS '):
            continue
        tokens = line.rstrip().split()
        sat = tokens[1]
        epoch = datetime.datetime(year   = int(tokens[2]),
                                  month  = int(tokens[3]),
                                  day    = int(tokens[4]),
                                  hour   = int(tokens[5]),
                                  minute = int(tokens[6]),
                                  second = int(float(tokens[7])),
                                  )
        if 'D' in tokens[9]:
            tokens[9] = tokens[9].replace('D', 'E')
        delta_tsv = float(tokens[9])
        SAT.append(sat)
        EPOCH.append(epoch)
        DELTA_TSV.append(delta_tsv)
    df = pd.DataFrame({
        'Epoch'    : EPOCH,
        'SatName'  : SAT,
        'DeltaTSV' : DELTA_TSV,
    })
    df = df[df['Epoch'] < (df['Epoch'].values[0] + pd.Timedelta(1, unit='day'))]
    df = df.reset_index(drop=True)
    return df

def read_sp3_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()        
    for index, line in enumerate(lines):
        if line.startswith('%c '):
            time_system = line.split()[3]
            assert((time_system == 'GPS') or (time_system == 'ccc'))
            continue
        if line.startswith('* '):
            start_index = index
            break
    lines = lines[start_index:]

    data = []
    for line in lines:
        if line.startswith('* '):
            tokens = line.rstrip().split()
            epoch = datetime.datetime(
                year   = int(tokens[1]),
                month  = int(tokens[2]),
                day    = int(tokens[3]),
                hour   = int(tokens[4]),
                minute = int(tokens[5]),
                second = int(float(tokens[6])),
            )
        elif line.startswith('P'):
            tokens = line.rstrip().split()
            sat = tokens[0][1:]
            x, y, z, delta_t = [float(s) for s in tokens[1:5]]
            x = x * 1e+3
            y = y * 1e+3
            z = z * 1e+3
            delta_t = delta_t * 1e-6
            data.append([epoch, sat, x, y, z, delta_t])
    columns = ['Epoch', 'SatName', 'X', 'Y', 'Z', 'DeltaTSV_SP3']
    df = pd.DataFrame(data, columns=columns)
    df = df[df['Epoch'] < (df['Epoch'].values[0] + pd.Timedelta(1, unit='day'))]
    df = df[~((df['X'] == 0) & (df['Y'] == 0) & (df['Z'] == 0))]
    df = df.reset_index(drop=True)
    return df

def read_SINEX_TRO_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    for index, line in enumerate(lines):
        if '+TROP/SOLUTION' in line:
            start_index = index + 2
            break
    lines = lines[start_index:]
    data = []
    for line in lines:
        if '-TROP/SOLUTION' in line:
            break
        tokens  = line.strip().split()
        y, d, s = [int(x) for x in tokens[1].split(':')]
        epoch = datetime.datetime(y+2000, 1, 1) + datetime.timedelta(days=d-1) + datetime.timedelta(seconds=s)
        data.append([epoch] + [1e-3 * float(x) for x in tokens[2:]])
    columns = ['Epoch',
               'TROTOT', 'TROTOT_STD',
               'TGNTOT', 'TGNTOT_STD',
               'TGETOT', 'TGETOT_STD']
    df = pd.DataFrame(data, columns=columns)
    df = df[df['Epoch'] < (df['Epoch'].values[0] + pd.Timedelta(1, unit='day'))]
    df = df.reset_index(drop=True)
    return df

def read_IONEX_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    kw = dict()
    #==============================
    # read header
    #==============================
    for index, line in enumerate(lines):
        tokens = line.strip().split()
        if 'EPOCH OF FIRST MAP' in line:
            kw['time_1'] = np.datetime64(datetime.datetime(
                year   = int(tokens[0]),
                month  = int(tokens[1]),
                day    = int(tokens[2]),
                hour   = int(tokens[3]),
                minute = int(tokens[4]),
                second = int(tokens[5]),
            ))
            continue
        if 'EPOCH OF LAST MAP' in line:
            kw['time_2'] = np.datetime64(datetime.datetime(
                year   = int(tokens[0]),
                month  = int(tokens[1]),
                day    = int(tokens[2]),
                hour   = int(tokens[3]),
                minute = int(tokens[4]),
                second = int(tokens[5]),                
            ))
            continue
        if 'INTERVAL' in line:
            kw['time_delta'] = np.timedelta64(datetime.timedelta(
                seconds=int(tokens[0]),
            ))
            continue
        if 'HGT1 / HGT2 / DHGT' in line:
            h1, h2, dh = [float(x) for x in tokens[0:3]]
            assert(h1 == h2)
            assert(dh == 0.0)
            kw['iono_height'] = h1 * 1000
            continue
        if 'LAT1 / LAT2 / DLAT' in line:
            lat_1, lat_2, lat_delta = [float(x) for x in tokens[0:3]]
            assert((lat_2 - lat_1) * lat_delta > 0)
            if (lat_1 > lat_2):
                flip_lat = True
                lat_1, lat_2 = lat_2, lat_1
                lat_delta = - lat_delta
            else:
                flip_lat = False
            kw['lat_1']     = np.deg2rad(lat_1)
            kw['lat_2']     = np.deg2rad(lat_2)
            kw['lat_delta'] = np.deg2rad(lat_delta)
            continue
        if 'LON1 / LON2 / DLON' in line:
            lng_1, lng_2, lng_delta = [float(x) for x in tokens[0:3]]
            assert((lng_2 - lng_1) * lng_delta > 0)
            if (lng_1 > lng_2):
                flip_lng = True
                lng_1, lng_2 = lng_2, lng_1
                lng_delta = - lng_delta
            else:
                flip_lng = False
            kw['lng_1']     = np.deg2rad(lng_1)
            kw['lng_2']     = np.deg2rad(lng_2)
            kw['lng_delta'] = np.deg2rad(lng_delta)
            continue
        if 'MAPPING FUNCTION' in line:
            assert(tokens[0] == 'COSZ')
            continue
        if 'BASE RADIUS' in line:
            kw['base_radius'] = 1000 * float(tokens[0])
            continue
        if 'EXPONENT' in line:
            TEC_coeff = 10**float(tokens[0])
            continue
        if 'MAP DIMENSION' in line:
            assert(int(tokens[0]) == 2)
            continue
        if 'END OF HEADER' in line:
            line_count = index + 1
            break
    #==============================
    # read data
    #==============================
    roundint = lambda x : int(round(x))
    N_lat  = 1 + roundint((kw['lat_2'] - kw['lat_1']) / kw['lat_delta'])
    N_lng  = 1 + roundint((kw['lng_2'] - kw['lng_1']) / kw['lng_delta'])
    N_time = 1 + roundint((kw['time_2'] - kw['time_1']) / kw['time_delta'])
    iono_map = np.zeros((N_time, N_lat, N_lng), dtype=np.float64)

    data_per_line = 16
    lines_per_data = (N_lng + data_per_line - 1) // data_per_line
    
    for time_count in range(N_time):
        assert('START OF TEC MAP' in lines[line_count])
        assert(int(lines[line_count].strip().split()[0]) == time_count + 1)
        line_count += 1
        assert('EPOCH OF CURRENT MAP' in lines[line_count])
        line_count += 1
        for lat_count in range(N_lat):
            assert('LAT/LON1/LON2/DLON/H' in lines[line_count])
            line_count += 1
            values = []
            for i in range(lines_per_data):
                values.extend([int(x) for x in lines[line_count+i].strip().split()])
            if 9999 in values:
                print('Warning: There is non-available TEC values.')
            iono_map[time_count, lat_count, :] = np.array(values).astype(float)
            line_count += lines_per_data
        assert('END OF TEC MAP' in lines[line_count])
        assert(int(lines[line_count].strip().split()[0]) == time_count + 1)
        line_count += 1

    if flip_lat:
        iono_map = np.flip(iono_map, axis=1)
    if flip_lng:
        iono_map = np.flip(iono_map, axis=2)
    iono_map = iono_map * TEC_coeff
    kw['iono_map']  = iono_map
    kw['lat_range'] = np.linspace(kw['lat_1'], kw['lat_2'], N_lat)
    kw['lng_range'] = np.linspace(kw['lng_1'], kw['lng_2'], N_lng)
    return IONEX(**kw)

In [ ]:
%%writefile transform.py
import numpy as np
from dataclasses import dataclass

import constants as C

@dataclass
class ECEF:
    x: np.array
    y: np.array
    z: np.array

    def to_numpy(self):
        return np.stack([self.x, self.y, self.z], axis=0)

    @staticmethod
    def from_numpy(pos):
        x, y, z = [np.squeeze(w) for w in np.split(pos, 3, axis=-1)]
        return ECEF(x=x, y=y, z=z)

@dataclass
class BLH:
    lat : np.array
    lng : np.array
    hgt : np.array

@dataclass
class ENU:
    east  : np.array
    north : np.array
    up    : np.array

@dataclass
class AZEL:
    elevation : np.array
    azimuth   : np.array
    zenith    : np.array

def BLH_to_ECEF(blh):
    a  = C.WGS84_SEMI_MAJOR_AXIS
    e2 = C.WGS84_SQUARED_FIRST_ECCENTRICITY
    sin_B = np.sin(blh.lat)
    cos_B = np.cos(blh.lat)
    sin_L = np.sin(blh.lng)
    cos_L = np.cos(blh.lng)
    n = a / np.sqrt(1 - e2*sin_B**2)
    x = (n + blh.hgt) * cos_B * cos_L
    y = (n + blh.hgt) * cos_B * sin_L
    z = ((1 - e2) * n + blh.hgt) * sin_B
    return ECEF(x=x, y=y, z=z)

def ECEF_to_BLH_approximate(ecef):
    a = C.WGS84_SEMI_MAJOR_AXIS
    b = C.WGS84_SEMI_MINOR_AXIS
    e2  = C.WGS84_SQUARED_FIRST_ECCENTRICITY
    e2_ = C.WGS84_SQUARED_SECOND_ECCENTRICITY
    x = ecef.x
    y = ecef.y
    z = ecef.z
    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(z * (a/b), r)
    B = np.arctan2(z + (e2_*b)*np.sin(t)**3, r - (e2*a)*np.cos(t)**3)
    L = np.arctan2(y, x)
    n = a / np.sqrt(1 - e2*np.sin(B)**2)
    H = (r / np.cos(B)) - n
    return BLH(lat=B, lng=L, hgt=H)

ECEF_to_BLH = ECEF_to_BLH_approximate

def ECEF_to_ENU(pos, base):
    dx = pos.x - base.x
    dy = pos.y - base.y
    dz = pos.z - base.z
    base_blh = ECEF_to_BLH(base)
    sin_B = np.sin(base_blh.lat)
    cos_B = np.cos(base_blh.lat)
    sin_L = np.sin(base_blh.lng)
    cos_L = np.cos(base_blh.lng)
    e = -sin_L*dx + cos_L*dy
    n = -sin_B*cos_L*dx - sin_B*sin_L*dy + cos_B*dz
    u =  cos_B*cos_L*dx + cos_B*sin_L*dy + sin_B*dz
    return ENU(east=e, north=n, up=u)

def ENU_to_AZEL(enu):
    e = enu.east
    n = enu.north
    u = enu.up
    elevation = np.arctan2(u, np.sqrt(e**2 + n**2))
    azimuth   = np.arctan2(e, n)
    zenith    = (0.5 * np.pi) - elevation
    return AZEL(elevation=elevation,
                azimuth=azimuth,
                zenith=zenith)

def ECEF_to_AZEL(pos, base):
    return ENU_to_AZEL(ECEF_to_ENU(pos, base))

def haversine_distance(blh_1, blh_2):
    dlat = blh_2.lat - blh_1.lat
    dlng = blh_2.lng - blh_1.lng
    a = np.sin(dlat/2)**2 + np.cos(blh_1.lat) * np.cos(blh_2.lat) * np.sin(dlng/2)**2
    dist = 2 * C.HAVERSINE_RADIUS * np.arcsin(np.sqrt(a))
    return dist

def hubenys_distance(blh_1, blh_2):
    Rx = C.WGS84_SEMI_MAJOR_AXIS
    Ry = C.WGS84_SEMI_MINOR_AXIS
    E2 = C.WGS84_SQUARED_FIRST_ECCENTRICITY
    num_M = Rx * (1 - E2)
    Dy = blh_1.lat - blh_2.lat
    Dx = blh_1.lng - blh_2.lng
    P  = 0.5 * (blh_1.lat + blh_2.lat)
    W  = np.sqrt(1 - E2 * np.sin(P)**2)
    M  = num_M / W**3
    N  = Rx / W
    d2 = (Dy * M)**2 + (Dx * N * np.cos(P))**2
    d  = np.sqrt(d2)
    return d

def jacobian_BLH_to_ECEF(blh):
    a  = C.WGS84_SEMI_MAJOR_AXIS
    e2 = C.WGS84_SQUARED_FIRST_ECCENTRICITY
    B = blh.lat
    L = blh.lng
    H = blh.hgt
    cos_B = np.cos(B)
    sin_B = np.sin(B)
    cos_L = np.cos(L)
    sin_L = np.sin(L)
    N = a / np.sqrt(1 - e2*sin_B**2)
    dNdB = a * e2 * sin_B * cos_B * (1 - e2*sin_B**2)**(-3/2)
    N_plus_H = N + H
    cos_B_cos_L = cos_B * cos_L
    cos_B_sin_L = cos_B * sin_L
    sin_B_cos_L = sin_B * cos_L
    sin_B_sin_L = sin_B * sin_L

    dXdB = dNdB*cos_B_cos_L - N_plus_H*sin_B_cos_L
    dYdB = dNdB*cos_B_sin_L - N_plus_H*sin_B_sin_L
    dZdB = (1-e2)*dNdB*sin_B + (1-e2)*N_plus_H*cos_B

    dXdL = - N_plus_H * cos_B_sin_L
    dYdL =   N_plus_H * cos_B_cos_L
    dZdL = np.zeros_like(dXdL)

    dXdH = cos_B_cos_L
    dYdH = cos_B_sin_L
    dZdH = sin_B

    J = np.stack([[dXdB, dXdL, dXdH],
                  [dYdB, dYdL, dYdH],
                  [dZdB, dZdL, dZdH]], axis=0)
    axes = list(range(2, J.ndim)) + [0, 1]
    J = np.transpose(J, axes)
    return J

def jacobian_ECEF_to_ENU(blh):
    B = blh.lat
    L = blh.lng
    cos_B = np.cos(B)
    sin_B = np.sin(B)
    cos_L = np.cos(L)
    sin_L = np.sin(L)
    
    dEdX = -sin_L
    dEdY =  cos_L
    dEdZ = np.zeros_like(dEdX)
    
    dNdX = -sin_B*cos_L
    dNdY = -sin_B*sin_L
    dNdZ =  cos_B

    dUdX = cos_B*cos_L
    dUdY = cos_B*sin_L
    dUdZ = sin_B

    J = np.stack([[dEdX, dEdY, dEdZ],
                  [dNdX, dNdY, dNdZ],
                  [dUdX, dUdY, dUdZ]], axis=0)
    axes = list(range(2, J.ndim)) + [0, 1]
    J = np.transpose(J, axes)
    return J

def pd_haversine_distance(df1, df2):
    blh1 = BLH(
        lat=np.deg2rad(df1['latDeg'].values),
        lng=np.deg2rad(df1['lngDeg'].values),
        hgt=0,
    )
    blh2 = BLH(
        lat=np.deg2rad(df2['latDeg'].values),
        lng=np.deg2rad(df2['lngDeg'].values),
        hgt=0,
    )
    return haversine_distance(blh1, blh2)

In [ ]:
%%writefile urls.py
import os
import datetime

from constants import GPS_ORIGIN_DAY

def clock_url(d):
    gps_week_number = (d - GPS_ORIGIN_DAY).days // 7
    days_of_year    = (d - datetime.date(d.year, 1, 1)).days + 1
    dirname = f'/gnss/products/mgex/{gps_week_number:04d}'
    if gps_week_number >= 2113:
        clk_filename = f'COD0MGXFIN_{d.year:04d}{days_of_year:03d}0000_01D_30S_CLK.CLK.gz'
    else:
        clk_filename = f'GRG0MGXFIN_{d.year:04d}{days_of_year:03d}0000_01D_30S_CLK.CLK.gz'
    return (dirname, clk_filename)

def orbit_url(d):
    gps_week_number = (d - GPS_ORIGIN_DAY).days // 7
    days_of_year    = (d - datetime.date(d.year, 1, 1)).days + 1
    dirname = f'/gnss/products/mgex/{gps_week_number:04d}'
    if gps_week_number >= 2113:
        sp3_filename = f'COD0MGXFIN_{d.year:04d}{days_of_year:03d}0000_01D_05M_ORB.SP3.gz'
    else:
        sp3_filename = f'GRG0MGXFIN_{d.year:04d}{days_of_year:03d}0000_01D_15M_ORB.SP3.gz'
    return (dirname, sp3_filename)

def ionosphere_url(d):
    year  = d.year
    year2 = year % 100
    days_of_year = (d - datetime.date(year, 1, 1)).days + 1
    dirname  = f'/gnss/products/ionex/{year:04d}/{days_of_year:03d}'
    filename = f'igsg{days_of_year:03d}0.{year2}i.Z'
    return (dirname, filename)

def troposphere_url(d):
    year  = d.year
    year2 = year % 100
    days_of_year = (d - datetime.date(year, 1, 1)).days + 1
    dirname  = f'/gnss/products/troposphere/zpd/{year:04d}/{days_of_year:03d}'
    filename = f'stfu{days_of_year:03d}0.{year2:02d}zpd.gz'
    return (dirname, filename)

def clock_filename(root, d):
    filename = os.path.splitext(clock_url(d)[1])[0]
    return os.path.join(root, filename)

def orbit_filename(root, d):
    filename = os.path.splitext(orbit_url(d)[1])[0]
    return os.path.join(root, filename)

def ionosphere_filename(root, d):
    filename = os.path.splitext(ionosphere_url(d)[1])[0]
    return os.path.join(root, filename)

def troposphere_filename(root, d):
    filename = os.path.splitext(troposphere_url(d)[1])[0]
    return os.path.join(root, filename)

In [ ]:
%%writefile delay_model.py
import numpy as np
import pandas as pd
from scipy.interpolate import RectBivariateSpline

import transform

def calc_iono_delay(ionex, epoch, carrier_freq, sat_azel, rec_blh):
    assert(epoch >= ionex.time_1)
    assert(epoch <= ionex.time_2)

    th_el = sat_azel.elevation
    th_az = sat_azel.azimuth
    
    R = ionex.base_radius
    H = ionex.iono_height
    z = np.arcsin((R / (R + H)) * np.cos(th_el))
    F = 1 / np.cos(z)
    
    psi = (0.5 * np.pi) - th_el - z
    sin_psi = np.sin(psi)
    cos_psi = np.cos(psi)
    sin_az  = np.sin(th_az)
    cos_az  = np.cos(th_az)

    sin_lat = np.sin(rec_blh.lat)
    cos_lat = np.cos(rec_blh.lat)
    
    IPP_lat = np.arcsin( sin_lat*cos_psi + cos_lat*sin_psi*cos_az )
    IPP_lng = rec_blh.lng + np.arcsin( sin_psi * sin_az / np.cos(IPP_lat) )

    index_0 = (epoch - ionex.time_1) // ionex.time_delta
    index_1 = index_0 + 1
    T0 = ionex.time_1 + ionex.time_delta * index_0
    T1 = T0 + ionex.time_delta
    alpha = (epoch - T0) / ionex.time_delta
    assert((0.0 <= alpha) and (alpha < 1.0))

    omega = np.pi / (12 * 3600)
    IPP_lng_0 = IPP_lng + omega * (epoch - T0).astype('timedelta64[s]').astype(int)
    IPP_lng_1 = IPP_lng + omega * (epoch - T1).astype('timedelta64[s]').astype(int)

    lat_range = ionex.lat_range
    lng_range = ionex.lng_range
    iono_map  = ionex.iono_map
    E_0 = RectBivariateSpline(lat_range, lng_range, iono_map[index_0, :, :], kx=1, ky=1, s=0)(IPP_lat, IPP_lng_0, grid=False)
    E_1 = RectBivariateSpline(lat_range, lng_range, iono_map[index_1, :, :], kx=1, ky=1, s=0)(IPP_lat, IPP_lng_1, grid=False)
    E_t = ((1.0 - alpha) * E_0) + (alpha * E_1)
    iono_delay = F * ((40.3 * 1e+16) / carrier_freq**2) * E_t
    return iono_delay

def calc_tropo_delay(sinex, epoch, sat_azel):
    th_el = sat_azel.elevation
    th_az = sat_azel.azimuth

    TIME = sinex['Epoch'].values
    time_delta = TIME[1] - TIME[0]
    index_0 = (epoch - TIME[0]) // time_delta
    index_1 = index_0 + 1
    alpha = (epoch - TIME[index_0]) / time_delta
    assert((0.0 <= alpha) and (alpha < 1.0))

    zpd_0 = sinex['TROTOT'].values[index_0]
    zpd_1 = sinex['TROTOT'].values[index_1]
    grad_n_0 = sinex['TGNTOT'].values[index_0]
    grad_n_1 = sinex['TGNTOT'].values[index_1]
    grad_e_0 = sinex['TGETOT'].values[index_0]
    grad_e_1 = sinex['TGETOT'].values[index_1]

    cos_az = np.cos(th_az)
    sin_az = np.sin(th_az)
    zpd_0  = zpd_0 + (grad_n_0 * cos_az) + (grad_e_0 * sin_az)
    zpd_1  = zpd_1 + (grad_n_1 * cos_az) + (grad_e_1 * sin_az)

    zpd_t = ((1 - alpha) * zpd_0) + (alpha * zpd_1)
    M = 1.001 / np.sqrt(0.002001 + np.sin(th_el)**2)
    return M * zpd_t

In [ ]:
%%writefile carrier_smoothing.py
import numpy as np
import pandas as pd
import scipy.sparse
import scipy.sparse.linalg

import constants as C

def bit_check(X, name):
    mask = 2**C.ACCUMULATED_DELTA_RANGE_STATE_BIT_INT_MAP[name]
    return (X & mask) != 0

def do_carrier_smoothing(gnss_df, beta=0.1):
    output_df_list = []
    t_ref = gnss_df['millisSinceGpsEpoch'].min()
    for (sat_name, sig_type), tmp_df in gnss_df.groupby(['SatName', 'SignalType']):
        time = 1e-3 * (tmp_df['millisSinceGpsEpoch'] - t_ref)
    
        clock_bias_ref  = tmp_df['FullBiasNanos'].values[0]
        clock_bias      = (tmp_df['FullBiasNanos'] - clock_bias_ref).values + tmp_df['BiasNanos'].values
        clock_drift_m   = (C.LIGHT_SPEED * 1e-9) * np.diff(clock_bias)

        raw_pseudo_range = tmp_df['rawPrM'].values

        accumurated_delta_range = tmp_df['AccumulatedDeltaRangeMeters'].values
        delta_range = np.diff(accumurated_delta_range) - clock_drift_m

        can_compare_to_next = bit_check(tmp_df['AccumulatedDeltaRangeState'].values, 'VALID')
        can_compare_to_prev = ( bit_check(tmp_df['AccumulatedDeltaRangeState'].values, 'VALID')
                                & (~bit_check(tmp_df['AccumulatedDeltaRangeState'].values, 'RESET'))
                                & (~bit_check(tmp_df['AccumulatedDeltaRangeState'].values, 'CYCLE_SLIP'))
                               )
        delta_range_valid = can_compare_to_next[0:-1] & can_compare_to_prev[1:]
        delta_range_valid = delta_range_valid & (np.diff(time) < 2.0)

        N = time.shape[0]
        A = scipy.sparse.eye(N)
        diag_B = delta_range_valid.astype(float) * np.full(N-1, beta**(-2))
        B = scipy.sparse.spdiags(diag_B, [0], N-1, N-1)
        D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)
        Q = A + (D.T @ B @ D)
        c = (A @ raw_pseudo_range) + (D.T @ (B @ delta_range))
        tmp_df['smoothedPrM'] = scipy.sparse.linalg.spsolve(Q, c)
        output_df_list.append(tmp_df)
    output_df = pd.concat(output_df_list, axis=0)
    output_df.sort_values(['millisSinceGpsEpoch', 'SatName', 'SignalType'], inplace=True, ignore_index=True)
    return output_df

In [ ]:
%%writefile smooth_height.py
import numpy as np
import scipy.sparse
import scipy.sparse.linalg

def calc_C(time_x, time_y, dt):
    N_x = time_x.shape[0]
    N_y = time_y.shape[0]
    x_index = np.floor(time_y / dt).astype(int)
    alpha   = (time_y / dt) - x_index
    C = np.empty(shape=(N_y, N_x), dtype=np.object)
    for i_x in range(N_x):
        C[0, i_x] = 0
    for i_y in range(N_y):
        i_x = x_index[i_y]
        C[i_y, i_x] = alpha[i_y]
        if i_x < N_x - 1:
            C[i_y, i_x+1] = 1.0 - alpha[i_y]
    C = scipy.sparse.bmat(C, format='csr')
    return C

def smooth_height_once(time_y, height_y, T_max, beta, dt):
    N_x   = int(np.ceil(T_max / dt) + 1)
    N_y   = time_y.shape[0]
    time_x  = dt * np.arange(N_x)
    C = calc_C(time_x, time_y, dt)
    B = scipy.sparse.spdiags(np.full(N_x-1, beta), [0], N_x-1, N_x-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N_x), np.ones(N_x)]), [0, 1], N_x-1, N_x)
    Q = (C.T @ C) + (D.T @ B @ D)
    c = C.T @ height_y
    height_x = scipy.sparse.linalg.spsolve(Q, c)
    height_y_star = C @ height_x
    sol = dict()
    sol['time_x']   = time_x
    sol['height_x'] = height_x
    sol['height_y_star'] = height_y_star
    return sol

def smooth_height(time_pred, height_pred, valid, T_max, beta, dt):
    time_y   = time_pred[valid].copy()
    height_y = height_pred[valid].copy()
    for loop in range(2):
        sol = smooth_height_once(time_y, height_y, T_max=T_max, beta=beta, dt=dt)
        valid = np.abs(height_y - sol['height_y_star']) < 30
        time_y   = time_y[valid].copy()
        height_y = height_y[valid].copy()
    C = calc_C(sol['time_x'], time_pred, dt)
    return C @ sol['height_x']

In [ ]:
%%writefile preprocess.py
import sys
import os
import glob
import itertools
import traceback
import multiprocessing
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline

import io_f
import urls
import constants as C

INPUT_PATH    = '../input/google-smartphone-decimeter-challenge'
SAT_DATA_PATH = '../input/gsdc-merged-clk-and-sp3'
DEST_PATH     = '_features'

CONSTE_ID_GPS = C.CONSTELLATION_TYPE_MAP['GPS']
CONSTE_ID_GLO = C.CONSTELLATION_TYPE_MAP['GLONASS']
CONSTE_ID_GAL = C.CONSTELLATION_TYPE_MAP['GALILEO']
CONSTE_ID_QZS = C.CONSTELLATION_TYPE_MAP['QZSS']
CONSTE_ID_BDS = C.CONSTELLATION_TYPE_MAP['BEIDOU']

def add_extra_features_1st(key, gnss_df, log):
    nanosSinceGpsEpoch = gnss_df['TimeNanos'] - gnss_df['FullBiasNanos']
    gnss_df['millisSinceGpsEpoch'] = nanosSinceGpsEpoch // 10**6
    gnss_df['Epoch'] = pd.to_timedelta(nanosSinceGpsEpoch, unit='ns') + C.GPS_ORIGIN_DATETIME

    N = gnss_df.shape[0]
    SatName    = np.empty(shape=(N, ), dtype=np.object)
    SignalType = np.empty(shape=(N, ), dtype=np.object)
    FixedSvid  = np.empty(shape=(N, ), dtype=np.int32)
    columns = ['ConstellationType', 'Svid', 'CarrierFrequencyHz']
    for index, (conste, svid, freq) in enumerate(gnss_df[columns].itertuples(index=False, name=None)):
        if (conste == CONSTE_ID_GPS) and abs(freq - C.FREQ_GPS_L1) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['GPS']
            signal_type = 'GPS_L1'
            assert((1 <= svid) and (svid <= 32))
            fixed_svid = svid
        elif (conste == CONSTE_ID_GPS) and abs(freq - C.FREQ_GPS_L5) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['GPS']
            signal_type = 'GPS_L5'
            assert((1 <= svid) and (svid <= 32))
            fixed_svid = svid
        elif (conste == CONSTE_ID_GAL) and abs(freq - C.FREQ_GAL_E1) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['GALILEO']
            signal_type = 'GAL_E1'
            assert((1 <= svid) and (svid <= 36))
            fixed_svid = svid
        elif (conste == CONSTE_ID_GAL) and abs(freq - C.FREQ_GAL_E5A) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['GALILEO']
            signal_type = 'GAL_E5A'
            assert((1 <= svid) and (svid <= 36))
            fixed_svid = svid
        elif (conste == CONSTE_ID_BDS) and abs(freq - C.FREQ_BDS_B1I) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['BEIDOU']
            signal_type = 'BDS_B1I'
            assert((1 <= svid) and (svid <= 61))
            fixed_svid = svid
        elif (conste == CONSTE_ID_QZS) and abs(freq - C.FREQ_QZS_J1) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['QZSS']
            signal_type = 'QZS_J1'
            fixed_svid = C.QZSS_PRN_SVID_MAP[svid]
        elif (conste == CONSTE_ID_QZS) and abs(freq - C.FREQ_QZS_J5) < C.FREQ_TOL:
            sat_name_prefix = C.SYSTEM_NAME_MAP['QZSS']
            signal_type = 'QZS_J5'
            fixed_svid = C.QZSS_PRN_SVID_MAP[svid]
        elif (conste == CONSTE_ID_GLO):
            sat_name_prefix = C.SYSTEM_NAME_MAP['GLONASS']
            signal_type = 'GLO_G1'
            if svid > 24:
                freq_channel = svid - 100
                fixed_svid   = C.GLONASS_FCN_OSN_MAP[key][str(svid)]
            else:
                freq_channel = C.GLONASS_FREQ_CHANNEL_MAP[svid]
                fixed_svid   = svid
            assert( (-7 <= freq_channel) and (freq_channel <= 6) )
            freq_nominal = C.FREQ_GLO_G1_NOMINAL + freq_channel * C.FREQ_GLO_G1_DELTA
            assert( abs(freq - freq_nominal) < C.FREQ_TOL )
        else:
            print((conste, svid, freq))
            raise RuntimeError('unknown signal type')
        SatName[index]    = f'{sat_name_prefix}{fixed_svid:02d}'
        SignalType[index] = signal_type
        FixedSvid[index]  = fixed_svid
        del sat_name_prefix, signal_type, fixed_svid
        pass
    gnss_df['SatName']    = SatName
    gnss_df['SignalType'] = SignalType
    gnss_df['FixedSvid']  = FixedSvid
    return gnss_df

def bit_check(X, name):
    mask = 2**C.RAW_STATE_BIT_INV_MAP[name]
    return (X & mask) != 0

def remove_invalid_measurements(df, log):
    state    = df['State'].values
    sig_type = df['SignalType'].values
    conste   = df['ConstellationType'].values

    is_gal_E1     = (sig_type == 'GAL_E1')
    is_non_gal_E1 = np.logical_not(is_gal_E1)
    code_lock_0   = is_non_gal_E1 & bit_check(state, 'Code Lock')
    code_lock_1   = is_gal_E1     & bit_check(state, 'Galileo E1BC Code Lock')
    code_lock_ok  = code_lock_0 | code_lock_1

    is_glo     = (conste == C.CONSTELLATION_TYPE_MAP['GLONASS'])
    is_nonglo  = np.logical_not(is_glo)
    time_of_0  = is_nonglo & bit_check(state, 'Time Of Week Decoded State') & bit_check(state, 'Time Of Week Known')
    time_of_1  = is_glo & bit_check(state, 'GLONASS Time Of Day Decoded') & bit_check(state, 'GLONASS Time Of Day Known')
    time_of_ok = time_of_0 | time_of_1

    msec_ambi_ok     = np.logical_not(bit_check(state, 'Millisecond Ambiguity'))
    sigma_rectime_ok = (df['ReceivedSvTimeUncertaintyNanos'] <= C.ReceivedSvTimeUncertaintyNanos_THRESHOLD).values
    cn0_ok           = (df['Cn0DbHz'] >= C.Cn0DbHz_THRESHOLD).values
    svid_ok          = (df['FixedSvid'] >= 1).values

    valid = (code_lock_ok
             & time_of_ok
             & msec_ambi_ok
             & sigma_rectime_ok
             & cn0_ok
             & svid_ok)
    df = df[valid]
    df = df.reset_index(drop=True)
    return df

def calc_rawPrM_GPS(df):
    week_number  = (df['Epoch'].dt.date - C.GPS_ORIGIN_DAY).astype('timedelta64[D]').astype(np.int64) // 7
    offset_nanos = week_number * 7 * 24 * 3600 * 10**9
    dt_nanos = (df['TimeNanos'] - df['FullBiasNanos'] - offset_nanos - df['ReceivedSvTimeNanos'])
    dt_nanos = dt_nanos + (dt_nanos < 0).astype(int) * 7 * 24 * 3600 * 10**9
    dt_nanos = dt_nanos - df['BiasNanos']
    return (C.LIGHT_SPEED * 1e-9) * dt_nanos.values

def calc_rawPrM_BDS(df):
    week_number  = (df['Epoch'].dt.date - C.GPS_ORIGIN_DAY).astype('timedelta64[D]').astype(np.int64) // 7
    offset_nanos = (week_number * 7 * 24 * 3600 + C.BEIDOU_LEAP_SECONDS) * 10**9
    dt_nanos = (df['TimeNanos'] - df['FullBiasNanos'] - offset_nanos - df['ReceivedSvTimeNanos'])
    dt_nanos = dt_nanos + (dt_nanos < 0).astype(int) * 7 * 24 * 3600 * 10**9
    dt_nanos = dt_nanos - df['BiasNanos']
    return (C.LIGHT_SPEED * 1e-9) * dt_nanos.values

def calc_rawPrM_GLO(df):
    msk_day = pd.to_datetime(df['Epoch'], utc=True).dt.tz_convert(C.TZ_MSK).dt.date
    msk_days_from_gps_origin = (msk_day - C.GPS_ORIGIN_DAY).astype('timedelta64[D]').astype(np.int64)
    offset_nanos = ((msk_days_from_gps_origin * 24 - 3) * 3600 + C.GLONASS_LEAP_SECONDS) * 10**9
    dt_nanos = (df['TimeNanos'] - df['FullBiasNanos'] - offset_nanos - df['ReceivedSvTimeNanos'])
    dt_nanos = dt_nanos + (dt_nanos < 0).astype(int) * 24 * 3600 * 10**9
    dt_nanos = dt_nanos - df['BiasNanos']
    return (C.LIGHT_SPEED * 1e-9) * dt_nanos.values

def add_raw_pseudo_range(df, log):
    is_GLO = (df['ConstellationType'] == CONSTE_ID_GLO)
    is_BDS = (df['ConstellationType'] == CONSTE_ID_BDS)
    df_GLO = df[is_GLO].copy()
    df_BDS = df[is_BDS].copy()
    df_GPS = df[~(is_GLO | is_BDS)].copy()
    df_GPS['rawPrM'] = calc_rawPrM_GPS(df_GPS)
    df_BDS['rawPrM'] = calc_rawPrM_BDS(df_BDS)
    df_GLO['rawPrM'] = calc_rawPrM_GLO(df_GLO)
    df = pd.concat([df_GPS, df_BDS, df_GLO], axis=0)
    df = df[df['rawPrM'] < C.RAW_PSEUDO_RANGE_THRESHOLD]
    df = df.sort_values(['millisSinceGpsEpoch', 'SatName', 'SignalType'])
    df = df.reset_index(drop=True)
    return df

def add_clock_bias_meter(raw_df, derived_df, log):
    output_df_list = []
    for (sat_name, sig_type), raw_sat_df in raw_df.groupby(['SatName', 'SignalType']):
        svid = int(sat_name[1:])
        derived_sat_df = derived_df[ (derived_df['signalType'] == sig_type) & (derived_df['svid'] == svid) ]
        if derived_sat_df.shape[0] <= 1:
            log.append(f'{sat_name} not in derived.csv.')
            continue
        t_ref = derived_sat_df['millisSinceGpsEpoch'].min()
        X_in  = 1e-3 * (derived_sat_df['millisSinceGpsEpoch'] - t_ref).values
        Y_in  = derived_sat_df['satClkBiasM'].values
        X_out = 1e-3 * (raw_sat_df['millisSinceGpsEpoch'] - t_ref).values
        Y_out = InterpolatedUnivariateSpline(X_in, Y_in, k=1, ext=3)(X_out)
        raw_sat_df['satClkBiasM'] = Y_out
        output_df_list.append(raw_sat_df)
    output_df = pd.concat(output_df_list, axis=0)
    output_df.sort_values(['millisSinceGpsEpoch', 'SatName', 'SignalType'], inplace=True, ignore_index=True)
    return output_df

def add_satellite_orbit(gnss_df, log):
    sp3_days = np.unique([(gnss_df['Epoch'].min() - C.ORBIT_TIME_MARGIN).date(),
                          (gnss_df['Epoch'].max() + C.ORBIT_TIME_MARGIN).date()])
    sp3_filelist = [os.path.join(SAT_DATA_PATH, d.strftime('SP3_%Y%m%d.csv')) for d in sp3_days]
    sp3_df_list = [pd.read_csv(f) for f in sp3_filelist]
    for df in sp3_df_list:
        df['Epoch'] = df['Epoch'].astype(np.datetime64)
    sp3_df = io_f.concat_sp3(sp3_df_list)

    t0 = gnss_df['Epoch'].min()

    gnss_sat_dfs = []
    for sat, gnss_sat_df in gnss_df.groupby('SatName'):
        sp3_sat_df = sp3_df[sp3_df['SatName'] == sat]
        if sp3_sat_df.shape[0] == 0:
            log.append(f'{sat} not available.')
            continue
        TIME_ref = (sp3_sat_df['Epoch'] - t0).astype(np.int64).values * 1e-9
        xPos_fn  = InterpolatedUnivariateSpline(TIME_ref, sp3_sat_df['X'].values, k=5)
        yPos_fn  = InterpolatedUnivariateSpline(TIME_ref, sp3_sat_df['Y'].values, k=5)
        zPos_fn  = InterpolatedUnivariateSpline(TIME_ref, sp3_sat_df['Z'].values, k=5)
        xVel_fn  = xPos_fn.derivative()
        yVel_fn  = yPos_fn.derivative()
        zVel_fn  = zPos_fn.derivative()

        TIME = (gnss_sat_df['Epoch'] - t0).astype(np.int64).values * 1e-9
        xPos = xPos_fn(TIME)
        yPos = yPos_fn(TIME)
        zPos = zPos_fn(TIME)
        xVel = xVel_fn(TIME)
        yVel = yVel_fn(TIME)
        zVel = zVel_fn(TIME)
        gnss_sat_df['xSatPosM']   = xPos
        gnss_sat_df['ySatPosM']   = yPos
        gnss_sat_df['zSatPosM']   = zPos
        gnss_sat_df['xSatVelMps'] = xVel
        gnss_sat_df['ySatVelMps'] = yVel
        gnss_sat_df['zSatVelMps'] = zVel
        gnss_sat_df['relClkBiasM'] = - (2 / C.LIGHT_SPEED) * (xPos*xVel + yPos*yVel + zPos*zVel)
        gnss_sat_dfs.append(gnss_sat_df)
        pass
    gnss_df = pd.concat(gnss_sat_dfs, axis=0)
    gnss_df = gnss_df.sort_values(['millisSinceGpsEpoch', 'SatName', 'SignalType'])
    gnss_df = gnss_df.reset_index(drop=True)
    return gnss_df

def remove_QZSS(df, log):
    if df[df['SignalType'] == 'QZS_J1'].shape[0] < 50:
        df = df[df['SignalType'] != 'QZS_J1']
    if df[df['SignalType'] == 'QZS_J5'].shape[0] < 50:
        df = df[df['SignalType'] != 'QZS_J5']
    df = df.sort_values(['millisSinceGpsEpoch', 'SatName', 'SignalType'])
    df = df.reset_index(drop=True)
    return df

def fix_derived_timestamp(df_raw, df_derived):
    raw_timestamps     = df_raw['millisSinceGpsEpoch'].unique()
    derived_timestamps = df_derived['millisSinceGpsEpoch'].unique()
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
    df_derived['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], df_derived['millisSinceGpsEpoch'])))

    df_derived = df_derived.drop_duplicates(['millisSinceGpsEpoch', 'constellationType', 'svid', 'signalType'])    
    df_derived = df_derived.sort_values('millisSinceGpsEpoch', ignore_index=True)
    
    return df_derived

def process_gnss_df(prefix, drive, phone, key, df):
    log = []
    df  = add_extra_features_1st(key, df, log)
    df['defaultIsrbM'] = df['SignalType'].map(C.PHONE_ISRBM[phone]).astype(float)
    df  = remove_invalid_measurements(df, log)
    df  = remove_QZSS(df, log)
    df  = add_raw_pseudo_range(df, log)
    
    derived_df = pd.read_csv(f'{INPUT_PATH}/{prefix}/{drive}/{phone}/{phone}_derived.csv')
    derived_df = fix_derived_timestamp(df, derived_df)
    df  = add_clock_bias_meter(df, derived_df, log)
    
    df  = add_satellite_orbit(df, log)
    log = [f'[{key:<50}] {msg}' for msg in log]
    return df, log

def do_preprocess(path):
    prefix, drive, phone = path.split('/')[-3:]
    key = f'{prefix}_{drive}_{phone}'
    df1 = io_f.read_GnssLog_Raw(f'{INPUT_PATH}/{prefix}/{drive}/{phone}/{phone}_GnssLog.txt')
    try:
        df2, log = process_gnss_df(prefix, drive, phone, key, df1)
        df2.to_csv(f'{DEST_PATH}/{key}.csv', index=False)
        N1  = df1.shape[0]
        N2  = df2.shape[0]
        percent = 100.0 * N2 / N1
        log.append(f'[{key:<50}] {N2}/{N1} ({percent:.1f})')
        return log
    except ValueError:
        traceback.print_exc(file=sys.stderr)
        return f'{key:<50}: Fail'
    pass

def main():
    os.makedirs(DEST_PATH, exist_ok=True)
    
    pathlist = sorted(glob.glob(f'{INPUT_PATH}/train/*/*') + glob.glob(f'{INPUT_PATH}/test/*/*'))

    processes = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=processes) as pool:
        results = pool.imap_unordered(do_preprocess, pathlist)
        results = tqdm(results, total=len(pathlist))
        results = itertools.chain.from_iterable(results)
        results = list(results)
    with open('_preprocess.log', 'w') as f:
        for msg in sorted(results):
            print(msg, file=f)
    return

In [ ]:
%%writefile positioning_stage1.py
import glob
import datetime
import sys
import os
import multiprocessing
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import io_f
import constants as C
import transform
import urls
import delay_model
import smooth_height
import carrier_smoothing

INPUT_PATH      = '../input/google-smartphone-decimeter-challenge'
DELAY_DATA_PATH = '../input/gsdc-external-data-for-delay-models'
FEATURE_PATH    = '_features'
DEST_PATH       = '_raw_gnss_stage1'

INIT_REC_POS = transform.BLH_to_ECEF(transform.BLH(lat=C.INIT_B, lng=C.INIT_L, hgt=C.INIT_H)).to_numpy()

def get_ionex(days):
    file_list  = [urls.ionosphere_filename(DELAY_DATA_PATH, d) for d in days]
    ionex_list = [io_f.read_IONEX_file(f) for f in file_list]
    ionex = io_f.concat_ionex(ionex_list)
    return ionex

def get_default_sinex(days):
    epoch = [datetime.datetime.combine(days[ 0], datetime.time( 0,  0)),
             datetime.datetime.combine(days[-1], datetime.time(23, 55))]
    df = pd.DataFrame({
        'Epoch' : epoch,
    })
    df['TROTOT']     = C.DEFAULT_TROPO_DELAY_M
    df['TROTOT_STD'] = 0.0
    df['TGNTOT']     = 0.0
    df['TGNTOT_STD'] = 0.0
    df['TGETOT']     = 0.0
    df['TGETOT_STD'] = 0.0
    return df

def get_sinex(days):
    file_list  = [urls.troposphere_filename(DELAY_DATA_PATH, d) for d in days]
    if not np.all([os.path.exists(f) for f in file_list]):
        return get_default_sinex(days)
    sinex_list = [io_f.read_SINEX_TRO_file(f) for f in file_list]
    sinex = pd.concat(sinex_list, axis=0)
    return sinex

def L2_norm(x):
    return np.sqrt(np.sum(x**2, axis=1))

def calc_azel(sat_pos, rec_pos):
    sat = transform.ECEF.from_numpy(sat_pos)
    rec = transform.ECEF.from_numpy(rec_pos)
    return transform.ECEF_to_AZEL(pos=sat, base=rec)

def calc_blh(pos):
    blh = transform.ECEF.from_numpy(pos)
    return transform.ECEF_to_BLH(blh)

def mask_to_weight(mask, epsilon=1e-2):
    return (1 - epsilon) * mask.astype(float) + epsilon

def solve_weight_least_square(WA, Wb, alpha, epsilon=1e-12):
    """
    J = ||WA x - Wb||^2 + alpha * ||x[4:]||^2
    """
    sys_A = WA.T @ WA
    sys_b = WA.T @ Wb
    N = WA.shape[1]
    for i in range(4, N):
        sys_A[i, i] += alpha
    u, s, vh = np.linalg.svd(sys_A, hermitian=True)
    cond = s[0] / s[-1]
    if cond * epsilon > 1:
        return
    sys_x = vh.T @ ((u.T @ sys_b) / s)
    return sys_x

def do_positioning_1epoch(name, ionex, sinex, epoch_df):
    ms_epoch = epoch_df['millisSinceGpsEpoch'].values[0]
    M = epoch_df.shape[0]
    sig_type     = epoch_df['SignalType'].values
    sig_type_set = set(sig_type) - set(['GPS_L1'])
    N_isbr       = len(sig_type_set)
    N_states     = N_isbr + 4
    if M < 4:
        print(f'[{name} {ms_epoch}] Too few satellites.')
        return
    A_t    = np.ones((M, 1))
    A_isbr = np.zeros((M, N_isbr))
    for i, s in enumerate(sorted(sig_type_set)):
        A_isbr[:, i] = (sig_type == s).astype(float)
    
    sat_pos_rec    = epoch_df[['xSatPosM', 'ySatPosM', 'zSatPosM']].values
    sat_vel_rec    = epoch_df[['xSatVelMps', 'ySatVelMps', 'zSatVelMps']].values
    raw_pr_m       = epoch_df['smoothedPrM'].values
    sat_clk_m      = epoch_df['satClkBiasM'].values
    default_isrb_m = epoch_df['defaultIsrbM'].values
    var_svtime_m   = (epoch_df['ReceivedSvTimeUncertaintyNanos'].values * (1e-9 * C.LIGHT_SPEED))**2
    carrier_freq   = epoch_df['CarrierFrequencyHz'].values
    epoch = epoch_df['Epoch'].values[0]

    sat_pos  = sat_pos_rec
    rec_pos  = INIT_REC_POS
    rec_time = 0.0
    outlier_mask = np.full(M, True)
    for outer_loop in range(3):
        success  = False
        for inner_loop in range(8):
            if inner_loop < 2:
                tof      = L2_norm(rec_pos - sat_pos) / C.LIGHT_SPEED
                sat_pos  = sat_pos_rec - (sat_vel_rec.T * (tof + rec_time)).T
                sat_azel = calc_azel(sat_pos, rec_pos)
                rec_blh  = calc_blh(rec_pos)
                sagnac_m = (C.OMEGA_EARTH / C.LIGHT_SPEED) * (sat_pos[:, 0] * rec_pos[1] - sat_pos[:, 1] * rec_pos[0])
                iono_delay_m  = delay_model.calc_iono_delay(ionex, epoch, carrier_freq, sat_azel, rec_blh)
                tropo_delay_m = delay_model.calc_tropo_delay(sinex, epoch, sat_azel)
                correct_pr_m  = raw_pr_m + sat_clk_m - iono_delay_m - tropo_delay_m - default_isrb_m - sagnac_m
                var_el_err_m  = (0.8 / np.maximum(0.05, np.sin(sat_azel.elevation)))**2
                el_mask       = (sat_azel.elevation > np.deg2rad(5.0))
                W = mask_to_weight(outlier_mask & el_mask) * (var_svtime_m + var_el_err_m)**(-0.5)
            pos_diff = rec_pos - sat_pos
            rho      = L2_norm(pos_diff)
            A_xyz    = ((1/rho) * pos_diff.T).T
            A  = np.concatenate([A_xyz, A_t, A_isbr], axis=1)
            b  = correct_pr_m - rho
            WA = (W * A.T).T
            Wb = W * b
            v = solve_weight_least_square(WA, Wb, alpha=(20.0)**(-2))
            if v is None:
                print(f'[{name} {ms_epoch}] Rank deficient.')
                return
            delta_pos = v[0:3]
            rec_time  = v[3] / C.LIGHT_SPEED
            rec_pos   = rec_pos + delta_pos
            if np.sqrt(np.sum(delta_pos**2)) < C.EPSILON_M:
                residual = b - (A @ v)
                outlier_mask = (residual < 80.0)
                success = True
                break
        if not success:
            print(f'[{name} {ms_epoch}] Did not converge.')
            return
    rec_blh = calc_blh(rec_pos)
    latDeg  = np.rad2deg(rec_blh.lat)
    lngDeg  = np.rad2deg(rec_blh.lng)
    height  = rec_blh.hgt
    isrbM   = (A_isbr @ v[4:]) + default_isrb_m
    log_df  = pd.DataFrame({
        'SatName'      : epoch_df['SatName'].values,
        'SignalType'   : sig_type,
        'xSatPosM'     : sat_pos[:, 0],
        'ySatPosM'     : sat_pos[:, 1],
        'zSatPosM'     : sat_pos[:, 2],
        'isrbM'        : isrbM,
        'ionoDelayM'   : iono_delay_m,
        'tropoDelayM'  : tropo_delay_m,
        'residualM'    : residual,
        'isOutlier'    : ~outlier_mask,
        'elevationDeg' : np.rad2deg(sat_azel.elevation),
        'azimuthDeg'   : np.rad2deg(sat_azel.azimuth),
    })
    log_df['millisSinceGpsEpoch'] = ms_epoch
    log_df['xRecPosM']    = rec_pos[0]
    log_df['yRecPosM']    = rec_pos[1]
    log_df['zRecPosM']    = rec_pos[2]
    log_df['recClkBiasM'] = v[3]
    return latDeg, lngDeg, height, log_df

def do_positioning(path):
    prefix, drive, phone, = path.split('/')[-3:]
    name = f'{prefix}_{drive}_{phone}'

    gnss_df  = pd.read_csv(f'{FEATURE_PATH}/{prefix}_{drive}_{phone}.csv')
    gnss_df['Epoch'] = gnss_df['Epoch'].astype(np.datetime64)
    gnss_df = carrier_smoothing.do_carrier_smoothing(gnss_df)

    days = np.unique([(gnss_df['Epoch'].min() - C.IONO_TIME_MARGIN).date(),
                      (gnss_df['Epoch'].max() + C.IONO_TIME_MARGIN).date()])
    ionex = get_ionex(days)
    sinex = get_sinex(days)

    data    = []
    log_dfs = []
    NaN = float('nan')
    for index, (epoch, epoch_df) in enumerate(gnss_df.groupby('millisSinceGpsEpoch')):
        result = do_positioning_1epoch(name, ionex, sinex, epoch_df)
        if result is not None:
            latDeg, lngDeg, height, log_df = result
            data.append((epoch, 1, latDeg, lngDeg, height))
            log_dfs.append(log_df)
        else:
            data.append((epoch, 0, NaN, NaN, NaN))
    columns = ['millisSinceGpsEpoch', 'valid', 'latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
    pred_df = pd.DataFrame(data, columns=columns).sort_values('millisSinceGpsEpoch')

    valid = (pred_df['valid'] == 1).values
    t_ref = pred_df['millisSinceGpsEpoch'].min()
    time_pred = 1e-3 * (pred_df['millisSinceGpsEpoch'] - t_ref).values
    height_pred = pred_df['heightAboveWgs84EllipsoidM'].values
    dt = 2.0
    T_max = np.max(time_pred)
    pred_df['height_smooth'] = smooth_height.smooth_height(
        time_pred=time_pred,
        height_pred=height_pred,
        valid=valid,
        T_max=T_max,
        beta=5.0,
        dt=dt,
    )
    
    pred_df.to_csv(f'{DEST_PATH}/{prefix}_{drive}_{phone}.csv', index=False)

    log_df = pd.concat(log_dfs, axis=0)
    log_df.to_csv(f'{DEST_PATH}/log_{prefix}_{drive}_{phone}.csv', index=False)

    return 
    
def main():
    os.makedirs(DEST_PATH, exist_ok=True)

    pathlist = sorted(glob.glob(f'{INPUT_PATH}/train/*/*') + glob.glob(f'{INPUT_PATH}/test/*/*'))

    processes = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=processes) as pool:
        results = pool.imap_unordered(do_positioning, pathlist)
        results = tqdm(results, total=len(pathlist))
        results = list(results)
    return

In [ ]:
%%writefile positioning_stage2.py
import glob
import datetime
import sys
import os
import multiprocessing
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import io_f
import constants as C
import transform
import urls
import delay_model
import carrier_smoothing

INPUT_PATH      = '../input/google-smartphone-decimeter-challenge'
DELAY_DATA_PATH = '../input/gsdc-external-data-for-delay-models'
FEATURE_PATH    = '_features'
PREV_STAGE_PATH = '_raw_gnss_stage1'
DEST_PATH       = '_raw_gnss_stage2'

INIT_REC_POS = transform.BLH_to_ECEF(transform.BLH(lat=C.INIT_B, lng=C.INIT_L, hgt=C.INIT_H)).to_numpy()

def get_ionex(days):
    file_list  = [urls.ionosphere_filename(DELAY_DATA_PATH, d) for d in days]
    ionex_list = [io_f.read_IONEX_file(f) for f in file_list]
    ionex = io_f.concat_ionex(ionex_list)
    return ionex

def get_default_sinex(days):
    epoch = [datetime.datetime.combine(days[ 0], datetime.time( 0,  0)),
             datetime.datetime.combine(days[-1], datetime.time(23, 55))]
    df = pd.DataFrame({
        'Epoch' : epoch,
    })
    df['TROTOT']     = C.DEFAULT_TROPO_DELAY_M
    df['TROTOT_STD'] = 0.0
    df['TGNTOT']     = 0.0
    df['TGNTOT_STD'] = 0.0
    df['TGETOT']     = 0.0
    df['TGETOT_STD'] = 0.0
    return df

def get_sinex(days):
    file_list  = [urls.troposphere_filename(DELAY_DATA_PATH, d) for d in days]
    if not np.all([os.path.exists(f) for f in file_list]):
        return get_default_sinex(days)
    sinex_list = [io_f.read_SINEX_TRO_file(f) for f in file_list]
    sinex = pd.concat(sinex_list, axis=0)
    return sinex

def L2_norm(x):
    return np.sqrt(np.sum(x**2, axis=1))

def calc_azel(sat_pos, rec_pos):
    sat = transform.ECEF.from_numpy(sat_pos)
    rec = transform.ECEF.from_numpy(rec_pos)
    return transform.ECEF_to_AZEL(pos=sat, base=rec)

def calc_blh(pos):
    blh = transform.ECEF.from_numpy(pos)
    return transform.ECEF_to_BLH(blh)

def mask_to_weight(mask, epsilon=1e-2):
    return (1 - epsilon) * mask.astype(float) + epsilon

def calc_height(rec_pos):
    ecef = transform.ECEF.from_numpy(rec_pos)
    blh  = transform.ECEF_to_BLH(ecef)    
    return np.array([blh.hgt])

def calc_height_gain(rec_pos, N_states):
    ecef = transform.ECEF.from_numpy(rec_pos)
    blh  = transform.ECEF_to_BLH(ecef)    
    J = transform.jacobian_BLH_to_ECEF(blh)
    Jinv = np.linalg.inv(J)
    A = np.zeros((1, N_states))
    A[0, 0:3] = Jinv[2, :]
    return A

def solve_weight_least_square(A1, b1, A2, b2, alpha, gamma, epsilon=1e-12):
    """
    J = ||A1 x - b1||^2 + alpha * ||x[4:]||^2 + gamma * ||A2 x - b2||^2
    """
    sys_A = (A1.T @ A1) + gamma * (A2.T @ A2)
    sys_b = (A1.T @ b1) + gamma * (A2.T @ b2)
    N = A1.shape[1]
    for i in range(4, N):
        sys_A[i, i] += alpha
    u, s, vh = np.linalg.svd(sys_A, hermitian=True)
    cond = s[0] / s[-1]
    if cond * epsilon > 1:
        return
    sys_x = vh.T @ ((u.T @ sys_b) / s)
    return sys_x

def do_positioning_1epoch(name, ionex, sinex, epoch_df, h0):
    ms_epoch = epoch_df['millisSinceGpsEpoch'].values[0]
    M = epoch_df.shape[0]
    sig_type     = epoch_df['SignalType'].values
    sig_type_set = set(sig_type) - set(['GPS_L1'])
    N_isbr       = len(sig_type_set)
    N_states     = N_isbr + 4
    if M < 3:
        print(f'[{name} {ms_epoch}] Too few satellites.')
        return
    A_t    = np.ones((M, 1))
    A_isbr = np.zeros((M, N_isbr))
    for i, s in enumerate(sorted(sig_type_set)):
        A_isbr[:, i] = (sig_type == s).astype(float)
    
    sat_pos_rec    = epoch_df[['xSatPosM', 'ySatPosM', 'zSatPosM']].values
    sat_vel_rec    = epoch_df[['xSatVelMps', 'ySatVelMps', 'zSatVelMps']].values
    raw_pr_m       = epoch_df['smoothedPrM'].values
    sat_clk_m      = epoch_df['satClkBiasM'].values
    default_isrb_m = epoch_df['defaultIsrbM'].values
    var_svtime_m   = (epoch_df['ReceivedSvTimeUncertaintyNanos'].values * (1e-9 * C.LIGHT_SPEED))**2
    carrier_freq   = epoch_df['CarrierFrequencyHz'].values
    epoch = epoch_df['Epoch'].values[0]

    sat_pos  = sat_pos_rec
    rec_pos  = INIT_REC_POS
    rec_time = 0.0
    outlier_mask = np.full(M, True)
    for outer_loop in range(3):
        success  = False
        for inner_loop in range(8):
            if inner_loop < 2:
                tof      = L2_norm(rec_pos - sat_pos) / C.LIGHT_SPEED
                sat_pos  = sat_pos_rec - (sat_vel_rec.T * (tof + rec_time)).T
                sat_azel = calc_azel(sat_pos, rec_pos)
                rec_blh  = calc_blh(rec_pos)
                sagnac_m = (C.OMEGA_EARTH / C.LIGHT_SPEED) * (sat_pos[:, 0] * rec_pos[1] - sat_pos[:, 1] * rec_pos[0])
                iono_delay_m  = delay_model.calc_iono_delay(ionex, epoch, carrier_freq, sat_azel, rec_blh)
                tropo_delay_m = delay_model.calc_tropo_delay(sinex, epoch, sat_azel)
                correct_pr_m  = raw_pr_m + sat_clk_m - iono_delay_m - tropo_delay_m - default_isrb_m - sagnac_m
                var_el_err_m  = (0.8 / np.maximum(0.05, np.sin(sat_azel.elevation)))**2
                el_mask       = (sat_azel.elevation > np.deg2rad(5.0))
                W = mask_to_weight(outlier_mask & el_mask) * (var_svtime_m + var_el_err_m)**(-0.5)
            pos_diff = rec_pos - sat_pos
            rho      = L2_norm(pos_diff)
            A_xyz    = ((1/rho) * pos_diff.T).T
            A  = np.concatenate([A_xyz, A_t, A_isbr], axis=1)
            b  = correct_pr_m - rho
            WA = (W * A.T).T
            Wb = W * b
            Ah = calc_height_gain(rec_pos, N_states)
            bh = h0 - calc_height(rec_pos)
            v = solve_weight_least_square(WA, Wb,
                                          Ah, bh,
                                          alpha=(5.0)**(-2),
                                          gamma=2.2**(-2))
            if v is None:
                print(f'[{name} {ms_epoch}] Rank deficient.')
                return
            delta_pos = v[0:3]
            rec_time  = v[3] / C.LIGHT_SPEED
            rec_pos   = rec_pos + delta_pos
            if np.sqrt(np.sum(delta_pos**2)) < C.EPSILON_M:
                residual = b - (A @ v)
                outlier_mask = (np.abs(residual) < 50.0)
                success = True
                break
        if not success:
            print(f'[{name} {ms_epoch}] Did not converge.')
            return
    rec_blh = calc_blh(rec_pos)
    latDeg  = np.rad2deg(rec_blh.lat)
    lngDeg  = np.rad2deg(rec_blh.lng)
    height  = rec_blh.hgt
    isrbM   = (A_isbr @ v[4:]) + default_isrb_m
    log_df  = pd.DataFrame({
        'SatName'      : epoch_df['SatName'].values,
        'SignalType'   : sig_type,
        'xSatPosM'     : sat_pos[:, 0],
        'ySatPosM'     : sat_pos[:, 1],
        'zSatPosM'     : sat_pos[:, 2],
        'isrbM'        : isrbM,
        'ionoDelayM'   : iono_delay_m,
        'tropoDelayM'  : tropo_delay_m,
        'residualM'    : residual,
        'isOutlier'    : ~outlier_mask,
        'elevationDeg' : np.rad2deg(sat_azel.elevation),
        'azimuthDeg'   : np.rad2deg(sat_azel.azimuth),
    })
    log_df['millisSinceGpsEpoch'] = ms_epoch
    log_df['xRecPosM']    = rec_pos[0]
    log_df['yRecPosM']    = rec_pos[1]
    log_df['zRecPosM']    = rec_pos[2]
    log_df['recClkBiasM'] = v[3]
    return latDeg, lngDeg, height, log_df

def do_positioning(path):
    prefix, drive, phone, = path.split('/')[-3:]
    name = f'{prefix}_{drive}_{phone}'

    gnss_df  = pd.read_csv(f'{FEATURE_PATH}/{prefix}_{drive}_{phone}.csv')
    gnss_df['Epoch'] = gnss_df['Epoch'].astype(np.datetime64)
    gnss_df = carrier_smoothing.do_carrier_smoothing(gnss_df)

    prev_df = pd.read_csv(f'{PREV_STAGE_PATH}/{prefix}_{drive}_{phone}.csv')

    log_df = pd.read_csv(f'{PREV_STAGE_PATH}/log_{prefix}_{drive}_{phone}.csv')
    isrbM_map = { 'GPS_L1' : 0.0 }
    for sig_type, sig_df in log_df.groupby('SignalType'):
        if sig_type != 'GPS_L1':
            sig_df = sig_df.drop_duplicates(subset='millisSinceGpsEpoch')
            q10 = sig_df['isrbM'].quantile(0.10)
            q90 = sig_df['isrbM'].quantile(0.90)
            sig_df = sig_df[(q10 <= sig_df['isrbM']) & (sig_df['isrbM'] <= q90)]
            isrbM_map[sig_type] = sig_df['isrbM'].mean()
    gnss_df['defaultIsrbM'] = gnss_df['SignalType'].map(isrbM_map)

    days = np.unique([(gnss_df['Epoch'].min() - C.IONO_TIME_MARGIN).date(),
                      (gnss_df['Epoch'].max() + C.IONO_TIME_MARGIN).date()])
    ionex = get_ionex(days)
    sinex = get_sinex(days)

    data    = []
    log_dfs = []
    NaN = float('nan')
    for index, (epoch, epoch_df) in enumerate(gnss_df.groupby('millisSinceGpsEpoch')):
        h0 = prev_df[prev_df['millisSinceGpsEpoch'] == epoch]['height_smooth'].values[0]

        result = do_positioning_1epoch(name, ionex, sinex, epoch_df, h0)
        if result is not None:
            latDeg, lngDeg, height, log_df = result
            data.append((epoch, 1, latDeg, lngDeg, height))
            log_dfs.append(log_df)
        else:
            data.append((epoch, 0, NaN, NaN, NaN))
    columns = ['millisSinceGpsEpoch', 'valid', 'latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
    df = pd.DataFrame(data, columns=columns).sort_values('millisSinceGpsEpoch')
    df.to_csv(f'{DEST_PATH}/{prefix}_{drive}_{phone}.csv', index=False)

    log_df = pd.concat(log_dfs, axis=0)
    log_df.to_csv(f'{DEST_PATH}/log_{prefix}_{drive}_{phone}.csv', index=False)

    return 
    
def main():
    os.makedirs(DEST_PATH, exist_ok=True)

    pathlist = sorted(glob.glob(f'{INPUT_PATH}/train/*/*') + glob.glob(f'{INPUT_PATH}/test/*/*'))

    processes = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=processes) as pool:
        results = pool.imap_unordered(do_positioning, pathlist)
        results = tqdm(results, total=len(pathlist))
        results = list(results)
    return

In [ ]:
%%writefile make_new_baseline.py
import glob
import numpy as np
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline

import transform

INPUT_PATH  = '../input/google-smartphone-decimeter-challenge'
PREDICTION_PATH = '_raw_gnss_stage2'

TRAIN_BASELINE_DF = pd.read_csv(f'{INPUT_PATH}/baseline_locations_train.csv')
TEST_BASELINE_DF  = pd.read_csv(f'{INPUT_PATH}/baseline_locations_test.csv')
BASELINE_DF = pd.concat([TRAIN_BASELINE_DF, TEST_BASELINE_DF], axis=0)
def get_baseline(drive, phone):
    df = BASELINE_DF[BASELINE_DF['phone'] == f'{drive}_{phone}'].copy()
    df.reset_index(drop=True, inplace=True)
    return df

def make_output_df(prefix, drive, phone):
    base_df = get_baseline(drive, phone)

    pred_df = pd.read_csv(f'{PREDICTION_PATH}/{prefix}_{drive}_{phone}.csv')
    pred_df = pred_df[pred_df['valid'] == 1]
    pred_df = pred_df.reset_index(drop=True)

    t_ref    = base_df['millisSinceGpsEpoch'].min()
    TIME_y   = 1e-3 * (base_df['millisSinceGpsEpoch'] - t_ref).values
    TIME_x   = 1e-3 * (pred_df['millisSinceGpsEpoch'] - t_ref).values
    latDeg_x = pred_df['latDeg'].values
    lngDeg_x = pred_df['lngDeg'].values
    latDeg_y = InterpolatedUnivariateSpline(TIME_x, latDeg_x, k=1)(TIME_y)
    lngDeg_y = InterpolatedUnivariateSpline(TIME_x, lngDeg_x, k=1)(TIME_y)
    interp_df = pd.DataFrame({
        'collectionName'      : base_df['collectionName'],
        'phoneName'           : base_df['phoneName'],
        'millisSinceGpsEpoch' : base_df['millisSinceGpsEpoch'],
        'latDeg' : latDeg_y,
        'lngDeg' : lngDeg_y,
        'phone'  : base_df['phone'],
    })
    # Replace outliers with host baseline
    distance = transform.pd_haversine_distance(interp_df, base_df)
    alpha = (distance < 50).astype(float)
    output_df = pd.DataFrame({
        'collectionName'      : base_df['collectionName'],
        'phoneName'           : base_df['phoneName'],
        'millisSinceGpsEpoch' : base_df['millisSinceGpsEpoch'],
        'latDeg' : alpha * interp_df['latDeg'] + (1 - alpha) * base_df['latDeg'],
        'lngDeg' : alpha * interp_df['lngDeg'] + (1 - alpha) * base_df['lngDeg'],
        'phone'  : base_df['phone'],
    })
    return output_df

def print_score(output_df):
    score_list = []
    for gid, phone_df in output_df.groupby('phone'):
        drive, phone = gid.split('_')
        gt_df = pd.read_csv(f'{INPUT_PATH}/train/{drive}/{phone}/ground_truth.csv')
        d = transform.pd_haversine_distance(phone_df, gt_df)
        score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])
        score_list.append(score)
    score = np.mean(score_list)
    print(f'train score: {score:.3f}')
    return

def main():
    df_list = []
    for phone in np.unique(TRAIN_BASELINE_DF['phone']):
        collection_name, phone_name = phone.split('_')
        df_list.append(make_output_df('train', collection_name, phone_name))
    output_df = pd.concat(df_list)
    output_df = output_df.sort_values(['phone', 'millisSinceGpsEpoch'])
    output_df.to_csv('raw_gnss_train.csv', index=False)
    print_score(output_df)

    df_list = []
    for phone in np.unique(TEST_BASELINE_DF['phone']):
        collection_name, phone_name = phone.split('_')
        df_list.append(make_output_df('test', collection_name, phone_name))
    output_df = pd.concat(df_list)
    output_df = output_df.sort_values(['phone', 'millisSinceGpsEpoch'])
    output_df.to_csv('raw_gnss_test.csv', index=False)
    
    columns = ['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']
    sub_df = output_df[columns]
    sub_df.to_csv('submission.csv', index=False)
    return

## main

In [ ]:
import generate_phone_isrbM
generate_phone_isrbM.main()

In [ ]:
import preprocess
preprocess.main()

In [ ]:
import positioning_stage1
positioning_stage1.main()

In [ ]:
import positioning_stage2
positioning_stage2.main()

In [ ]:
import make_new_baseline
make_new_baseline.main()